In [2]:
# default_exp metrics

In [3]:
#export
import torch.nn.utils as nn_utils
from fastai.torch_basics import *
from fastai.data.all import *
from fastai.basics import *
from fastai.metrics import *
from dataclasses import field,asdict
from typing import List,Any,Dict,Callable
from collections import deque
import gym
from torch.optim import *

from fastrl.data import *
from fastrl.async_data import *
from fastrl.basic_agents import *
from fastrl.learner import *
from fastai.callback.progress import *
from fastrl.ptan_extension import *

import ptan

if IN_NOTEBOOK:
    from IPython import display
    import PIL.Image

/opt/conda/envs/fastrl/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729047590/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


# RL Metrics

> Metrics for tracking the progress of reinforcement learning agents.

In [5]:
# export
class AvgEpisodeRewardMetric(Metric):
    def __init__(self,experience_cls:ExperienceFirstLast):
        self.experience_cls=experience_cls
        self.rolling_rewards=deque([0],maxlen=100)
        
    def accumulate(self,learn):
#         yb=learn.yb
#         print(len(yb),len(yb[0]),yb)
        if type(learn.yb[0][0])!=ExperienceFirstLast:
            yb=[]
            for i in range(len(learn.xb[0])):
    #             print(learn.yb)
                yb.append(self.experience_cls(learn.xb[0][i].cpu().detach(),*(learn.yb[j][i].cpu().detach() for j in range(len(learn.yb)))))
        else:
            yb=learn.yb[0]
#             print(yb[-1])
#         yb=[ExperienceFirstLast([0],*(yb[k][i] for k in range(len(yb)))) for i in range(len(yb[0]))]
#         yb=[for yb.items()]
#         print([o.done for o in yb if o.done])
        if len([float(o.episode_reward) for o in yb if o.done and int(o.episode_reward)!=0])==0:return
#         print([o.episode_reward for o in yb if o.done])
        r=np.average([float(o.episode_reward) for o in yb if o.done and int(o.episode_reward)!=0])
#         print([y for y in yb if y.absolute_end])
#         for r in rewards:
        if r!=0:self.rolling_rewards.extend([r])
#         print(len(rewards))
#         if len(rewards)!=0:self.r=sum(rewards)/len(rewards)
        
    @property
    def value(self):return np.mean(self.rolling_rewards) if len(self.rolling_rewards)!=1 else self.rolling_rewards[0]
    @property
    def name(self):return 'avg_episode_r'

In [6]:
from fastrl.actorcritic.a3c_data import *

In [ ]:
env='CartPole-v1'

block=AsyncExperienceBlock(
    experience_block=partial(FirstLastExperienceBlock,a=0,seed=0,dls_kwargs={'bs':1,'n_steps':4,'num_workers':0,
                                                                             'verbose':False,'indexed':True,'shuffle_train':False}),
    n_processes=1,
    n=128
)
blk=IterableDataBlock(blocks=(block),
                      splitter=FuncSplitter(lambda x:False),
#                       batch_tfms=lambda x:(x[0],x),
                     )
dls=blk.dataloaders([env]*1,bs=3)

model=LinearA2C((4,),2)
agent=ActorCriticAgent(model=model)
learner=A3CLearner(dls,agent=agent,cbs=[A3CTrainer],reward_steps=4,
                   metrics=[AvgEpisodeRewardMetric()])
learner.fit(3)

starting


Process DataFitProcess-1:
Traceback (most recent call last):
  File "/opt/conda/envs/fastrl/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/envs/fastrl/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/project/fastrl/fastrl/async_data.py", line 43, in template_data_fit
    ifnone(pipe_out,noopo).send('Hello')
NameError: name 'pipe_out' is not defined


In [ ]:
# hide
from nbdev.export2html import *
from nbdev.export import *
notebook2script()
notebook2html()